In [1]:
library(asreml)
library(tidyverse)
require(openxlsx)

Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 4.2.2"


Online License checked out Sun Dec 25 02:25:20 2022


Warning message:
"package 'tidyverse' was built under R version 4.2.2"
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand()  masks Matrix::expand()
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()
✖ tidyr::pack()    masks Matrix::pack()
✖ tidyr::unpack()  masks Matrix::unpack()
Loading required package: openxlsx

Warning message:
"package 'openxlsx' was built under R version 4.2.2"


### Input data

In [2]:
df_norm <- read.csv("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-归一化.csv")
df_stat <- read.csv("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-描述性统计.csv")

df_raw <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3.xlsx")
df_raw_anno <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3.xlsx", sheet = 2)

df_ykl <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-源库流和抗逆分类.xlsx")
df_ykl_anno <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-源库流和抗逆分类.xlsx", sheet = 2)

df_stress <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-源库流和抗逆分类.xlsx", sheet = 3)
df_stress_anno <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V3-源库流和抗逆分类.xlsx", sheet = 4)


In [3]:
# Traits needs to be removed in V4
remove <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状汇总表V4-updated-20221202.xlsx", sheet = 3)
rm_traits <- remove$Remove_Trait

update_data <- function(df, rm_traits){
    df <- df %>% select(setdiff(colnames(.) ,rm_traits))
    return(df)
}
df_raw <- update_data(df_raw, rm_traits)
df_norm <- update_data(df_norm, rm_traits)
df_ykl <- update_data(df_ykl, rm_traits)
df_stress <- update_data(df_stress, rm_traits)

df_raw_anno <- df_raw_anno %>% mutate(性状分类1 = gsub(' ', '', 性状分类1))

In [4]:
pheno_sum <- read.xlsx("../phenotype_data/383农艺性状汇总表V3/383农艺性状 - 属性解释表V2 - 李洪戈.xlsx") %>% 
    mutate(性状分类1 = gsub(' ', '', 性状分类1)) %>% 
    subset(!性状 %in% rm_traits) %>% # remove problem traits
    mutate(Treatment = ifelse(is.na(Treatment), 'BK', Treatment)) %>% 
    mutate(Develop.period = ifelse(is.na(Develop.period), 'BK', Develop.period)) %>% 
    mutate(TreatDev = paste0(Treatment, '_', Develop.period)) %>% 
    mutate(Replicate2 = ifelse(is.na(Replicate), 'R1', Replicate)) %>% 
    mutate(Replicate2 = ifelse(Trait %in% c('FW_Leaf', 'FW_Stem', 'FW_Root', 'CLAM'), 'R1', Replicate2)) %>%  # Four records have AV but no replicate, change them to R1
    subset(Replicate2 != 'AV') # remove AV records

In [11]:
pheno_sum %>% filter(!is.na(Additional_info)) %>% dim

[1] 163  12

In [6]:
# generate dimension of each trait
generate_LocYearDevRep_num <- function(pheno_sum, trait, TraitDev){
    tmp <- pheno_sum %>% subset(Trait == trait & TreatDev == TraitDev)
    Loc_num = tmp$Location %>% unique %>% length
    Year_num = tmp$Year %>% unique %>% length
    Dev_num = tmp$Develop.period %>% unique %>% length
    Rep_num = tmp$Replicate2 %>% unique %>% length
    return(c(trait, TraitDev, Loc_num, Year_num, Dev_num, Rep_num))
}

LYR_num_list <- list()
for (trait in pheno_sum$Trait %>% unique){
    for (TreatDev in pheno_sum %>% subset(Trait == trait) %>% .$TreatDev %>% unique) {
        result_vec <- generate_LocYearDevRep_num(pheno_sum, trait, TreatDev)
        LYR_num_list[[paste0(trait, '_', TreatDev)]] = result_vec
    }
}

df_LYR <- do.call(rbind, LYR_num_list) %>% 
data.frame() %>% 
mutate(numCombo = paste0(X3, X4, X6)) %>% 
setNames(c('Trait', 'TreatDev', 'Location', 'Year', 'Development', 'Replicate', 'numCombo'))

In [10]:
df_LYR %>% head

,Trait,TreatDev,Location,Year,Development,Replicate,numCombo
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
FE_BK_BK,FE,BK_BK,5,3,1,2,532
FE_DR_BK,FE,DR_BK,3,2,1,2,322
FL_BK_BK,FL,BK_BK,5,3,1,2,532
FL_DR_BK,FL,DR_BK,3,2,1,2,322
FS_BK_BK,FS,BK_BK,5,3,1,2,532
FS_DR_BK,FS,DR_BK,3,2,1,2,322


### Funcition
可以做遗传相关性状，至少要有两个records
- numCombo = 111 不能做
- 生成data后，根据T1 T2 是否都有值 判定是否向下进行
- 根据因子水平判断，生成拟合公式

In [12]:
df_LYR %>% filter(numCombo != '111') %>% dim

[1] 596   7

In [17]:
# total bi-trait comparison
(596 * 596 -596) / 2

[1] 177310

In [13]:
generate_rgp <- function(pheno_sum, df_raw, traitA, treat_devA, traitB, treat_devB){
    generate_trait_df <- function(pheno_sum, df_raw, trait, treat_dev){
        tmp <- pheno_sum %>% # generate trait info
            subset(Trait == trait & TreatDev == treat_dev) %>% 
            subset(Replicate2 != 'AV')

        tmp <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
            gather(性状, value, 2:ncol(.)) %>% 
            merge(tmp, '性状') %>% 
            .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
            mutate(ID = as.factor(ID)) %>% 
            mutate(Trait = paste0(Trait, '_', TreatDev)) %>% 
            mutate(Trait = as.factor(Trait)) %>% 
            mutate(Location = as.factor(Location)) %>% 
            mutate(Year = as.factor(Year)) %>% 
            mutate(TreatDev = as.factor(TreatDev)) %>% 
            mutate(Replicate2 = as.factor(Replicate2)) %>% 
            mutate(value = as.numeric(value)) %>% 
            select(-性状) %>% 
            distinct(Trait, TreatDev, Location, Year, Replicate2, ID, .keep_all = TRUE)
        return(tmp)
    }
    
    # generate input
    tmp1 <- generate_trait_df(pheno_sum, df_raw, traitA, treat_devA)
    tmp2 <- generate_trait_df(pheno_sum, df_raw, traitB, treat_devB)
    data <- rbind(tmp1, tmp2) %>% 
        pivot_wider(names_from=Trait, values_from=value) %>% 
        set_names(colnames(.) %>% .[1:5] %>% c(., 'T1', 'T2')) %>% 
        subset(!is.na(T1) & !is.na(T2))
    
    if (nrow(data) != 0){
        
    L = levels(data$Location) %>% length
    Y = levels(data$Year) %>% length
    R = levels(data$Replicate2) %>% length
    
    if (L > 1 & Y >1 & R >1){
        eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location + trait:Year
    }else if (L <= 1 & Y >1 & R >1){
        eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Year
    }else if (L > 1 & Y <= 1 & R >1){
        eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location
    }else if (L > 1 & Y > 1 & R <=1){
        eq = cbind(T1, T2) ~ trait + trait:Location + trait:Year
    }else if (L <= 1 & Y <= 1 & R >1){
        eq = cbind(T1, T2) ~ trait + trait:Replicate2
    }else if (L <= 1 & Y > 1 & R <= 1){
        eq = cbind(T1, T2) ~ trait + trait:Year
    }else if (L > 1 & Y <= 1 & R <= 1){
        eq = cbind(T1, T2) ~ trait + trait:Location
    }else{
        print(paste(paste0(traitA, "_", treat_devA), 
                    paste0(traitB, "_", treat_devB),
                    "The LocYearRep combo is 111!", 
                    sep="\t"))
    }
        
    # rgp
    require(asreml)
    mod = asreml(eq,
                random = ~ us(trait):ID,
                residual = ~ units:us(trait),
                data=data)

    mod$converge
    
    rg.vp <- vpredict(mod,rg ~ V2/sqrt(V1*V3))
    rp.vp <- vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

    #遗传相关和表型相关的显著性
    moda = asreml(eq,
                random = ~ diag(trait):ID,
                residual = ~ units:us(trait),
                data=data)

    modb = asreml(eq,
                random = ~ diag(trait):ID,
                residual = ~ units:diag(trait),
                data=data)

    moda.p <- lrt.asreml(mod,moda) # 遗传相关显著性
    modb.p <- lrt.asreml(mod,modb) # 表型相关显著性

    rg.sig <- -log10(moda.p$`Pr(Chisq)`)
    rp.sig <- -log10(modb.p$`Pr(Chisq)`)

    rgp.str <- paste(paste0(traitA, "_", treat_devA), 
                     paste0(traitB, "_", treat_devB),
                     "rg", rg.vp$Estimate, rg.vp$SE, moda.p$df, moda.p$`LR-statistic`, moda.p$`Pr(Chisq)`,
                     rg.sig, 
                       "rp", rp.vp$Estimate, rp.vp$SE, modb.p$df, modb.p$`LR-statistic`, modb.p$`Pr(Chisq)`,
                     rp.sig, 
                     paste0(L, Y, R),
                       sep="\t")

    }else{
        rgp.str <- paste(paste0(traitA, "_", treat_devA), 
                         paste0(traitB, "_", treat_devB),
                         "No common records!", 
                         sep="\t")
    }
    return(rgp.str)
}

In [18]:
# # test for loop run
# test <- df_LYR %>% filter(numCombo != '111') %>% head
# test

# trait_vec <- test$Trait
# treatdev_vec <- test$TreatDev
# trait_num <- nrow(test)

# for (i in 1:(trait_num - 1)) {
#     for (j in (i + 1):trait_num) {
#         print(paste(trait_vec[i], treatdev_vec[i], "---", trait_vec[j], treatdev_vec[j]))
#         result <- generate_rgp(pheno_sum, df_raw, trait_vec[i], treatdev_vec[i],
#             trait_vec[j], treatdev_vec[j])
#         cat(result, file = "./ASreml_results/test_out.txt", append = T, sep = "\n")
#     }
# }

#### 1st run all

In [1]:
# run all results
test <- df_LYR %>% filter(numCombo != '111') 

trait_vec <- test$Trait
treatdev_vec <- test$TreatDev
trait_num <- nrow(test)

for (i in 1:(trait_num - 1)) {
    for (j in (i + 1):trait_num) {
        tryCatch({
            print(paste(trait_vec[i], treatdev_vec[i], "---", trait_vec[j], treatdev_vec[j]))
            result <- generate_rgp(pheno_sum, df_raw, trait_vec[i], treatdev_vec[i],
                trait_vec[j], treatdev_vec[j])
            cat(result, file = "./ASreml_results/run_all_25Dec2022.txt", append = T, sep = "\n")
        }, error = function(e) {
            cat("ERROR :", conditionMessage(e), "")
        })
    }
}

ERROR: Error in df_LYR %>% filter(numCombo != "111"): could not find function "%>%"


In [210]:
generate_rgp_debug <- function(pheno_sum, df_raw, traitA, treat_devA, traitB, treat_devB){
    generate_trait_df <- function(pheno_sum, df_raw, trait, treat_dev){
        tmp <- pheno_sum %>% # generate trait info
            subset(Trait == trait & TreatDev == treat_dev) %>% 
            subset(Replicate2 != 'AV')

        tmp <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
            gather(性状, value, 2:ncol(.)) %>% 
            merge(tmp, '性状') %>% 
            .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
            mutate(ID = as.factor(ID)) %>% 
            mutate(Trait = paste0(Trait, '_', TreatDev)) %>% 
            mutate(Trait = as.factor(Trait)) %>% 
            mutate(Location = as.factor(Location)) %>% 
            mutate(Year = as.factor(Year)) %>% 
            mutate(TreatDev = as.factor(TreatDev)) %>% 
            mutate(Replicate2 = as.factor(Replicate2)) %>% 
            mutate(value = as.numeric(value)) %>% 
            select(-性状) %>% 
            distinct(Trait, TreatDev, Location, Year, Replicate2, ID, .keep_all = TRUE)
        return(tmp)
    }
    
    # generate input
    tmp1 <- generate_trait_df(pheno_sum, df_raw, traitA, treat_devA)
    tmp2 <- generate_trait_df(pheno_sum, df_raw, traitB, treat_devB)
    data <- rbind(tmp1, tmp2) %>% 
        pivot_wider(names_from=Trait, values_from=value) %>% 
        set_names(colnames(.) %>% .[1:5] %>% c(., 'T1', 'T2')) %>% 
        subset(!is.na(T1) & !is.na(T2))
    
#     if (nrow(data) != 0){
        
#     L = levels(data$Location) %>% length
#     Y = levels(data$Year) %>% length
#     R = levels(data$Replicate2) %>% length
    
#     if (L > 1 & Y >1 & R >1){
#         eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location + trait:Year
#     }else if (L <= 1 & Y >1 & R >1){
#         eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Year
#     }else if (L > 1 & Y <= 1 & R >1){
#         eq = cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location
#     }else if (L > 1 & Y > 1 & R <=1){
#         eq = cbind(T1, T2) ~ trait + trait:Location + trait:Year
#     }else if (L <= 1 & Y <= 1 & R >1){
#         eq = cbind(T1, T2) ~ trait + trait:Replicate2
#     }else if (L <= 1 & Y > 1 & R <= 1){
#         eq = cbind(T1, T2) ~ trait + trait:Year
#     }else if (L > 1 & Y <= 1 & R <= 1){
#         eq = cbind(T1, T2) ~ trait + trait:Location
#     }else{
#         print(paste(paste0(traitA, "_", treat_devA), 
#                     paste0(traitB, "_", treat_devB),
#                     "The LocYearRep combo is 111!", 
#                     sep="\t"))
#     }
        
#     # rgp
#     require(asreml)
#     mod = asreml(eq,
#                 random = ~ us(trait):ID,
#                 residual = ~ units:us(trait),
#                 data=data)

#     mod$converge
    
#     rg.vp <- vpredict(mod,rg ~ V2/sqrt(V1*V3))
#     rp.vp <- vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

#     #遗传相关和表型相关的显著性
#     moda = asreml(eq,
#                 random = ~ diag(trait):ID,
#                 residual = ~ units:us(trait),
#                 data=data)

#     modb = asreml(eq,
#                 random = ~ diag(trait):ID,
#                 residual = ~ units:diag(trait),
#                 data=data)

#     moda.p <- lrt.asreml(mod,moda) # 遗传相关显著性
#     modb.p <- lrt.asreml(mod,modb) # 表型相关显著性

#     rg.sig <- -log10(moda.p$`Pr(Chisq)`)
#     rp.sig <- -log10(modb.p$`Pr(Chisq)`)

#     rgp.str <- paste(paste0(traitA, "_", treat_devA), 
#                      paste0(traitB, "_", treat_devB),
#                        "rg", rg.vp$Estimate, rg.vp$SE, moda.p$df, moda.p$`LR-statistic`, moda.p$`Pr(Chisq)`,
#                      rg.sig, 
#                        "rp", rp.vp$Estimate, rp.vp$SE, modb.p$df, modb.p$`LR-statistic`, modb.p$`Pr(Chisq)`,
#                      rp.sig,
#                        sep="\t")

#     }else{
#         rgp.str <- paste(paste0(traitA, "_", treat_devA), 
#                          paste0(traitB, "_", treat_devB),
#                          "No common records!", 
#                          sep="\t")
#     }
#     return(rgp.str)
    return(data)
}

In [211]:
generate_rgp_debug(pheno_sum, df_raw, 'FE', 'BK_BK','FE', 'DR_BK')

TreatDev,Location,Year,Replicate2,ID,T1,T2
<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
BK_BK,AL,18,R1,K001,9.8,NA
BK_BK,AL,18,R1,K002,9.5,NA
BK_BK,AL,18,R1,K003,9.2,NA
BK_BK,AL,18,R1,K004,10.0,NA
BK_BK,AL,18,R1,K005,7.4,NA
BK_BK,AL,18,R1,K006,8.4,NA
BK_BK,AL,18,R1,K007,9.3,NA
BK_BK,AL,18,R1,K008,6.6,NA
BK_BK,AL,18,R1,K009,10.6,NA


In [ ]:
phe

In [185]:
# generate_rgp <- function(pheno_sum, df_raw, traitA, treat_devA, traitB, treat_devB){
#     generate_trait_df <- function(pheno_sum, df_raw, trait, treat_dev){
#         tmp <- pheno_sum %>% # generate trait info
#             subset(Trait == trait & TreatDev == treat_dev) %>% 
#             subset(Replicate2 != 'AV')

#         tmp <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
#             gather(性状, value, 2:ncol(.)) %>% 
#             merge(tmp, '性状') %>% 
#             .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
#             mutate(ID = as.factor(ID)) %>% 
#             mutate(Trait = as.factor(Trait)) %>% 
#             mutate(Location = as.factor(Location)) %>% 
#             mutate(Year = as.factor(Year)) %>% 
#             mutate(TreatDev = as.factor(TreatDev)) %>% 
#             mutate(Replicate2 = as.factor(Replicate2)) %>% 
#             mutate(value = as.numeric(value)) %>% 
#             select(-性状) %>% 
#             distinct(Trait, TreatDev, Location, Year, Replicate2, ID, .keep_all = TRUE)
#         return(tmp)
#     }
    
#     # generate input
#     tmp1 <- generate_trait_df(pheno_sum, df_raw, traitA, treat_devA)
#     tmp2 <- generate_trait_df(pheno_sum, df_raw, traitB, treat_devB)
#     data <- rbind(tmp1, tmp2) %>% 
#         pivot_wider(names_from=Trait, values_from=value) %>% 
#         set_names(colnames(.) %>% .[1:5] %>% c(., 'T1', 'T2')) %>% 
#         subset(!is.na(T1) & !is.na(T2))

# #     # rgp
# #     require(asreml)
# #     mod = asreml(cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
# #                 random = ~ us(trait):ID,
# #                 residual = ~ units:us(trait),
# #                 data=data)

# #     mod$converge
    
# #     rg.vp <- vpredict(mod,rg ~ V2/sqrt(V1*V3))
# #     rp.vp <- vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

# #     #遗传相关和表型相关的显著性
# #     moda = asreml(cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
# #                 random = ~ diag(trait):ID,
# #                 residual = ~ units:us(trait),
# #                 data=data)

# #     modb = asreml(cbind(T1, T2) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
# #                 random = ~ diag(trait):ID,
# #                 residual = ~ units:diag(trait),
# #                 data=data)

# #     moda.p <- lrt.asreml(mod,moda) # 遗传相关显著性
# #     modb.p <- lrt.asreml(mod,modb) # 表型相关显著性

# #     rg.sig <- -log10(moda.p$`Pr(Chisq)`)
# #     rp.sig <- -log10(modb.p$`Pr(Chisq)`)

# #     rgp.str <- paste(paste0(traitA, "_", treat_devA), 
# #                      paste0(traitB, "_", treat_devB),
# #                        "rg", rg.vp$Estimate, rg.vp$SE, moda.p$df, moda.p$`LR-statistic`, moda.p$`Pr(Chisq)`,
# #                      rg.sig, 
# #                        "rp", rp.vp$Estimate, rp.vp$SE, modb.p$df, modb.p$`LR-statistic`, modb.p$`Pr(Chisq)`,
# #                      rp.sig,
# #                        sep="\t")

#     return(data)   
# }

In [179]:
# multiple vs 111
generate_rgp(pheno_sum, df_raw, 'FE', 'BK_BK', 'CFW', 'BK_30d')

# 111 vs 111
generate_rgp(pheno_sum, df_raw, 'SFW', 'BK_30d','CFW', 'BK_30d')

# mult vs mult
generate_rgp(pheno_sum, df_raw, 'FE', 'BK_BK','FL', 'BK_BK')


# mult vs unexpected
generate_rgp(pheno_sum, df_raw, 'FE', 'BK_BK','VWDM', 'BK_BK')

[1] "FE_BK_BK\tCFW_BK_30d\tNo common records!"

In [128]:
tmp3 %>% select(-性状) %>% distinct(Trait, TreatDev, Location, Year, Replicate2, ID,  .keep_all = TRUE)

Trait,TreatDev,Location,Year,Replicate2,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
VWDM,BK_BK,AL,18,R1,K001,90.91
VWDM,BK_BK,AL,18,R1,K002,80.00
VWDM,BK_BK,AL,18,R1,K003,63.64
VWDM,BK_BK,AL,18,R1,K004,30.00
VWDM,BK_BK,AL,18,R1,K005,90.00
VWDM,BK_BK,AL,18,R1,K006,40.00
VWDM,BK_BK,AL,18,R1,K007,60.00
VWDM,BK_BK,AL,18,R1,K008,20.00
VWDM,BK_BK,AL,18,R1,K009,66.67


In [91]:
rbind(tmp1, tmp3 %>% distinct())

Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K001,9.8
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K002,9.5
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K003,9.2
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K004,10.0
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K005,7.4
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K006,8.4
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K007,9.3
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K008,6.6
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K009,10.6


### Test run

In [10]:
# test data
tmp <- pheno_sum %>% # generate trait info
    subset(Trait == 'FE' & TreatDev == 'BK_BK') %>% 
    subset(Replicate2 != 'AV')

tmp1 <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
    gather(性状, value, 2:ncol(.)) %>% 
    merge(tmp, '性状') %>% 
    .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
    mutate(ID = as.factor(ID)) %>% 
    mutate(Trait = as.factor(Trait)) %>% 
    mutate(Location = as.factor(Location)) %>% 
    mutate(Year = as.factor(Year)) %>% 
    mutate(TreatDev = as.factor(TreatDev)) %>% 
    mutate(Replicate2 = as.factor(Replicate2)) %>% 
    mutate(value = as.numeric(value))

tmp1

Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K001,9.8
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K002,9.5
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K003,9.2
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K004,10.0
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K005,7.4
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K006,8.4
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K007,9.3
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K008,6.6
FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K009,10.6


In [12]:
# test data
tmp <- pheno_sum %>% # generate trait info
    subset(Trait == 'FL' & TreatDev == 'BK_BK') %>% 
    subset(Replicate2 != 'AV')

tmp2 <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
    gather(性状, value, 2:ncol(.)) %>% 
    merge(tmp, '性状') %>% 
    .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
    mutate(ID = as.factor(ID)) %>% 
    mutate(Trait = as.factor(Trait)) %>% 
    mutate(Location = as.factor(Location)) %>% 
    mutate(Year = as.factor(Year)) %>% 
    mutate(TreatDev = as.factor(TreatDev)) %>% 
    mutate(Replicate2 = as.factor(Replicate2)) %>% 
    mutate(value = as.numeric(value))

tmp2

Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K001,31.25
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K002,28.91
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K003,28.07
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K004,29.48
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K005,31.76
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K006,34.04
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K007,29.71
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K008,34.68
FL,BK_BK,AL,18,R1,FL_AL_18_CK_R1,K009,31.32


In [16]:
# test data
tmp <- pheno_sum %>% # generate trait info
    subset(Trait == 'VWDM' & TreatDev == 'BK_BK') %>% 
    subset(Replicate2 != 'AV')

tmp3 <- df_raw[c('ID', tmp$性状)] %>% # extract trait value
    gather(性状, value, 2:ncol(.)) %>% 
    merge(tmp, '性状') %>% 
    .[c('Trait', 'TreatDev', 'Location', 'Year', 'Replicate2', '性状', 'ID', 'value' )] %>% 
    mutate(ID = as.factor(ID)) %>% 
    mutate(Trait = as.factor(Trait)) %>% 
    mutate(Location = as.factor(Location)) %>% 
    mutate(Year = as.factor(Year)) %>% 
    mutate(TreatDev = as.factor(TreatDev)) %>% 
    mutate(Replicate2 = as.factor(Replicate2)) %>% 
    mutate(value = as.numeric(value))

tmp3

Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K001,90.91
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K002,80.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K003,63.64
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K004,30.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K005,90.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K006,40.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K007,60.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K008,20.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K009,66.67


In [170]:
data <- rbind(tmp1, tmp2) %>% 
    select(-性状) %>% 
    pivot_wider(names_from=Trait, values_from=value)

In [171]:
data %>% head

TreatDev,Location,Year,Replicate2,ID,FE,FL
<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
BK_BK,AL,18,R1,K001,9.8,31.25
BK_BK,AL,18,R1,K002,9.5,28.91
BK_BK,AL,18,R1,K003,9.2,28.07
BK_BK,AL,18,R1,K004,10.0,29.48
BK_BK,AL,18,R1,K005,7.4,31.76
BK_BK,AL,18,R1,K006,8.4,34.04


In [175]:
eq = cbind(FE, FL) ~ trait + trait:Replicate2 + trait:Location + trait:Year
mod = asreml(eq,
            random = ~ us(trait):ID,
            residual = ~ units:us(trait),
            data=data)

mod$converge
rg_result <- vpredict(mod, rg ~ V2/sqrt(V1 * V3)) %>% unlist
rp_result <- vpredict(mod, rp ~ (V2 + V6)/sqrt((V1 + V5) * (V3 + V7)))

# summary(mod)$varcomp

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sun Dec 25 01:15:02 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10759.85           1.0  14334 01:15:02    0.0
 2      -9753.06           1.0  14334 01:15:02    0.0
 3      -9328.43           1.0  14334 01:15:02    0.0
 4      -9158.77           1.0  14334 01:15:02    0.0
 5      -9088.01           1.0  14334 01:15:02    0.0 (1 restrained)
 6      -9074.25           1.0  14334 01:15:02    0.0
 7      -9073.19           1.0  14334 01:15:02    0.0
 8      -9073.18           1.0  14334 01:15:02    0.0


[1] TRUE

In [172]:
mod = asreml(cbind(FE, FL) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
            random = ~ us(trait):ID,
            residual = ~ units:us(trait),
            data=data)

mod$converge
rg_result <- vpredict(mod, rg ~ V2/sqrt(V1 * V3)) %>% unlist
rp_result <- vpredict(mod, rp ~ (V2 + V6)/sqrt((V1 + V5) * (V3 + V7)))

# summary(mod)$varcomp

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sun Dec 25 01:14:23 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10759.85           1.0  14334 01:14:23    0.0
 2      -9753.06           1.0  14334 01:14:23    0.0
 3      -9328.43           1.0  14334 01:14:23    0.0
 4      -9158.77           1.0  14334 01:14:23    0.0
 5      -9088.01           1.0  14334 01:14:23    0.0 (1 restrained)
 6      -9074.25           1.0  14334 01:14:23    0.0
 7      -9073.19           1.0  14334 01:14:23    0.0
 8      -9073.18           1.0  14334 01:14:23    0.0


[1] TRUE

In [71]:
rg.vp <- vpredict(mod,rg ~ V2/sqrt(V1*V3))
##计算表型相关
rp.vp <- vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

#遗传相关和表型相关的显著性
moda = asreml(cbind(FE, FL) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
            random = ~ diag(trait):ID,
            residual = ~ units:us(trait),
            data=data)

modb = asreml(cbind(FE, FL) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
            random = ~ diag(trait):ID,
            residual = ~ units:diag(trait),
            data=data)

moda.p <- lrt.asreml(mod,moda) # 遗传相关显著性
modb.p <- lrt.asreml(mod,modb) # 表型相关显著性

rg.sig <- -log10(moda.p$`Pr(Chisq)`)
rp.sig <- -log10(modb.p$`Pr(Chisq)`)

rgp.str <- paste(paste0("FE","_", "FL"), 
                   "rg", rg.vp$Estimate, rg.vp$SE, moda.p$df, moda.p$`LR-statistic`, moda.p$`Pr(Chisq)`,
                 rg.sig, 
                   "rp", rp.vp$Estimate, rp.vp$SE, modb.p$df, modb.p$`LR-statistic`, modb.p$`Pr(Chisq)`,
                 rp.sig,
                   sep="\t")

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 23:33:50 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10762.68           1.0  14334 23:33:50    0.0
 2      -9811.74           1.0  14334 23:33:50    0.0
 3      -9334.87           1.0  14334 23:33:50    0.0
 4      -9160.51           1.0  14334 23:33:50    0.0
 5      -9088.37           1.0  14334 23:33:50    0.0
 6      -9074.36           1.0  14334 23:33:50    0.0
 7      -9073.36           1.0  14334 23:33:50    0.0
 8      -9073.35           1.0  14334 23:33:50    0.0
Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 23:33:50 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10734.25           1.0  14334 23:33:50    0.0
 2      -9801.14           1.0  14334 23:33:50    0.0
 3      -9324.45           1.0  14334 23:33:50    0.0
 4      -9159.40           1.0  14334 23:33:50    0.0
 5      -9093.41           1.0  14334 23:33:50    0.0
 6      -9081.05    

In [72]:
rgp.str 

[1] "FE_FL\trg\t-0.0346210328327552\t0.0602822095174906\t1\t0.329228061000322\t0.283056993818661\t0.548126110027479\trp\t0.0172309020875404\t0.0183998792261941\t2\t14.1087415592483\t0.000302176532292142\t3.51973926697177"

In [47]:
#
mod1 = asreml(cbind(FE, FL) ~ trait + trait:Replicate2,
         random = ~ us(trait):ID,
         residual = ~ units:us(trait),
         data=data)

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 22:41:15 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -17301.87           1.0  14346 22:41:15    0.0
 2     -16639.29           1.0  14346 22:41:15    0.0
 3     -15992.03           1.0  14346 22:41:15    0.0
 4     -15660.40           1.0  14346 22:41:15    0.0
 5     -15554.94           1.0  14346 22:41:15    0.0
 6     -15540.36           1.0  14346 22:41:15    0.0
 7     -15539.34           1.0  14346 22:41:15    0.0
 8     -15539.33           1.0  14346 22:41:15    0.0


In [46]:
mod2 = asreml(cbind(FE, FL) ~ trait + trait:Replicate2 + trait:Location + trait:Year,
         random = ~ us(trait):ID,
         residual = ~ units:us(trait),
         data=data)

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 22:24:08 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10759.85           1.0  14334 22:24:08    0.0
 2      -9753.06           1.0  14334 22:24:08    0.0
 3      -9328.43           1.0  14334 22:24:08    0.0
 4      -9158.77           1.0  14334 22:24:09    0.0
 5      -9088.01           1.0  14334 22:24:09    0.0 (1 restrained)
 6      -9074.25           1.0  14334 22:24:09    0.0
 7      -9073.19           1.0  14334 22:24:09    0.0
 8      -9073.18           1.0  14334 22:24:09    0.0


In [49]:
mod3 = asreml(cbind(FE, FL) ~ trait + trait:Location/Replicate2 + trait:Year,
         random = ~ us(trait):ID,
         residual = ~ units:us(trait),
         data=data)

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 22:43:02 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10769.22           1.0  14326 22:43:02    0.0
 2      -9760.80           1.0  14326 22:43:02    0.0
 3      -9335.62           1.0  14326 22:43:02    0.0
 4      -9165.94           1.0  14326 22:43:02    0.0
 5      -9095.20           1.0  14326 22:43:02    0.0 (1 restrained)
 6      -9081.44           1.0  14326 22:43:02    0.0
 7      -9080.38           1.0  14326 22:43:02    0.0
 8      -9080.37           1.0  14326 22:43:02    0.0


In [58]:
mod4 = asreml(cbind(FE, FL) ~ trait + trait:Location + trait:Location/Replicate2 + trait:Year,
         random = ~ us(trait):ID,
         residual = ~ units:us(trait),
         data=data)

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 22:56:56 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -10769.22           1.0  14326 22:56:56    0.0
 2      -9760.80           1.0  14326 22:56:56    0.0
 3      -9335.62           1.0  14326 22:56:56    0.0
 4      -9165.94           1.0  14326 22:56:56    0.0
 5      -9095.20           1.0  14326 22:56:56    0.0 (1 restrained)
 6      -9081.44           1.0  14326 22:56:56    0.0
 7      -9080.38           1.0  14326 22:56:56    0.0
 8      -9080.37           1.0  14326 22:56:56    0.0


In [54]:
return_rg_rp <- function(mod){
    a = mod$converge
    b = vpredict(mod,rg ~ V2/sqrt(V1*V3))
    c = vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))
    return(list(a, b, c))
}


In [59]:
return_rg_rp(mod1)
return_rg_rp(mod2)
return_rg_rp(mod3)
return_rg_rp(mod4)

,Estimate,SE
,<dbl>,<dbl>
rg,-0.5195338,0.3751674
,Estimate,SE
,<dbl>,<dbl>
rp,0.2784736,0.01218926


,Estimate,SE
,<dbl>,<dbl>
rg,-0.03468058,0.06028148
,Estimate,SE
,<dbl>,<dbl>
rp,0.01773093,0.01840658


,Estimate,SE
,<dbl>,<dbl>
rg,-0.03468058,0.06028148
,Estimate,SE
,<dbl>,<dbl>
rp,0.01773093,0.01840658


,Estimate,SE
,<dbl>,<dbl>
rg,-0.03468058,0.06028148
,Estimate,SE
,<dbl>,<dbl>
rp,0.01773093,0.01840658


In [50]:
mod$converge
summary(mod)$varcomp

[1] TRUE

,component,std.error,z.ratio,bound,%ch
,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
trait:ID!trait_FE:FE,0.01096969,0.01316628,0.8331653,P,0
trait:ID!trait_FL:FE,-0.07976012,0.03299049,-2.4176700,P,0
trait:ID!trait_FL:FL,2.14857117,0.16831449,12.7652179,P,0
units:trait!R,1.00000000,NA,NA,F,0
units:trait!trait_FE:FE,3.12226541,0.05358227,58.2704975,P,0
units:trait!trait_FL:FE,1.23219719,0.04182012,29.4642221,P,0
units:trait!trait_FL:FL,3.31747719,0.05693306,58.2697889,P,0


In [41]:
vpredict(mod,rg ~ V2/sqrt(V1*V3))

,Estimate,SE
,<dbl>,<dbl>
rg,-0.5195338,0.3751674


In [42]:
vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

,Estimate,SE
,<dbl>,<dbl>
rp,0.2784736,0.01218926


In [33]:
rbind(tmp1, tmp3) %>% 
select(-性状) %>% 
pivot_wider(names_from=Trait, values_from=value)

Warning message:
"Values from `value` are not uniquely identified; output will contain list-cols.
* Use `values_fn = list` to suppress this warning.
* Use `values_fn = {summary_fun}` to summarise duplicates.
* Use the following dplyr code to identify duplicates.
  {data} %>%
    dplyr::group_by(TreatDev, Location, Year, Replicate2, ID, Trait) %>%
    dplyr::summarise(n = dplyr::n(), .groups = "drop") %>%
    dplyr::filter(n > 1L)"


TreatDev,Location,Year,Replicate2,ID,FE,VWDM
<fct>,<fct>,<fct>,<fct>,<fct>,<list>,<list>
BK_BK,AL,18,R1,K001,9.8,"90.91, 60.00, 70.97, 60.00"
BK_BK,AL,18,R1,K002,9.5,"80.00, 80.00, 76.67, 70.00"
BK_BK,AL,18,R1,K003,9.2,"63.64, 60.00, 58.06, 50.00"
BK_BK,AL,18,R1,K004,10,"30.00, 40.00, 36.67, 40.00"
BK_BK,AL,18,R1,K005,7.4,"90.00, 90.00, 83.33, 70.00"
BK_BK,AL,18,R1,K006,8.4,"40.00, 30.00, 33.33, 30.00"
BK_BK,AL,18,R1,K007,9.3,"60.00, 50.00, 53.33, 50.00"
BK_BK,AL,18,R1,K008,6.6,"20, 20, 20, 20"
BK_BK,AL,18,R1,K009,10.6,"66.67, 80.00, 64.29, 44.44"


In [35]:
?pivot_wider

In [34]:
tmp3

Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K001,90.91
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K002,80.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K003,63.64
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K004,30.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K005,90.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K006,40.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K007,60.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K008,20.00
VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K009,66.67


In [31]:
rbind(tmp1, tmp3) %>% 
select(-性状)

Trait,TreatDev,Location,Year,Replicate2,ID,value
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
FE,BK_BK,AL,18,R1,K001,9.8
FE,BK_BK,AL,18,R1,K002,9.5
FE,BK_BK,AL,18,R1,K003,9.2
FE,BK_BK,AL,18,R1,K004,10.0
FE,BK_BK,AL,18,R1,K005,7.4
FE,BK_BK,AL,18,R1,K006,8.4
FE,BK_BK,AL,18,R1,K007,9.3
FE,BK_BK,AL,18,R1,K008,6.6
FE,BK_BK,AL,18,R1,K009,10.6


In [30]:
tmp3 %>% head

,Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
1,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K001,90.91
2,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K002,80.00
3,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K003,63.64
4,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K004,30.00
5,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K005,90.00
6,VWDM,BK_BK,AL,18,R1,VWDM_AL_18_L_R1,K006,40.00


In [29]:
tmp1 %>% head

,Trait,TreatDev,Location,Year,Replicate2,性状,ID,value
,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<fct>,<dbl>
1,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K001,9.8
2,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K002,9.5
3,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K003,9.2
4,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K004,10.0
5,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K005,7.4
6,FE,BK_BK,AL,18,R1,FE_AL_18_CK_R1,K006,8.4


In [ ]:
tmp1 %>% merge()

In [21]:
df_raw %>% select(starts_with("FE"))

,FE_AL_18_CK_R1,FE_AL_18_CK_R2,FE_AL_18_CK_AV,FE_AL_19_CK_R1,FE_AL_19_CK_R2,FE_AL_19_CK_AV,FE_AY_18_R1,FE_AY_18_R2,FE_AY_18_AV,FE_AY_19_CK_R1,⋯,FE_DH_19_DR_AV,FE_DH_20_DR_R2,FE_DH_20_DR_R1,FE_DH_20_DR_AV,FE_AL_20_DR_R2,FE_AL_20_DR_R1,FE_AL_20_DR_AV,FE_XAU_20_DR_AV,FE_XAU_20_DR_R1,FE_XAU_20_DR_R2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9.8,8.7,9.25,7.1,7.0,7.05,7.2,8.0,7.60,6.8,⋯,0.99,0.96,0.97,0.96,0.9855072,0.9565217,0.9710145,1.0000000,1.0000000,NA
2,9.5,10.2,9.85,6.8,6.7,6.75,8.8,8.9,8.85,6.7,⋯,1.02,0.94,0.94,0.94,0.9275362,1.0151515,0.9703704,1.0232558,1.0153846,1.0312500
3,9.2,8.1,8.65,6.7,6.6,6.65,8.5,9.6,9.05,6.6,⋯,0.98,0.93,0.97,0.95,0.9552239,1.0000000,0.9772727,0.9923077,0.9692308,NA
4,10.0,8.8,9.40,6.6,6.0,6.30,7.7,8.6,8.15,6.7,⋯,1.06,0.94,0.94,0.94,0.9701493,0.9850746,0.9776119,1.0381679,1.0149254,NA
5,7.4,7.2,7.30,6.6,6.3,6.45,6.9,6.9,6.90,6.8,⋯,0.98,0.97,0.96,0.96,0.9850746,1.0000000,0.9924242,0.9846154,0.9692308,1.0000000
6,8.4,7.2,7.80,6.0,6.7,6.35,7.3,6.5,6.90,6.8,⋯,0.93,0.94,0.96,0.95,0.9696970,1.0151515,0.9924242,1.0465116,1.0312500,1.0615385
7,9.3,8.4,8.85,6.4,6.5,6.45,7.4,7.0,7.20,6.6,⋯,1.07,0.96,0.93,0.94,0.9701493,1.0000000,0.9847328,1.0078125,0.9846154,1.0317460
8,6.6,6.6,6.60,6.8,6.3,6.55,6.4,6.5,6.45,6.8,⋯,1.01,0.99,0.99,0.99,0.9411765,1.0000000,0.9701493,1.0075758,1.0151515,1.0000000
9,10.6,9.1,9.85,6.9,6.8,6.85,8.4,8.0,8.20,6.9,⋯,0.96,0.94,0.96,0.95,1.0000000,1.0000000,1.0000000,1.0074074,1.0000000,1.0149254


### Test code

In [4]:
library(learnasreml)

In [3]:
data("animalmodel.dat")
data("animalmodel.ped")
head(animalmodel.dat)
head(animalmodel.ped)

,ANIMAL,MOTHER,BYEAR,SEX,BWT,TARSUS
,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
1,1029,1145,968,1,10.77,24.77
2,1299,811,968,1,9.30,22.46
3,643,642,970,2,3.98,12.89
4,1183,1186,970,1,5.39,20.47
5,1238,1237,970,2,12.12,0.00
6,891,895,970,1,0.00,0.00


,ID,FATHER,MOTHER
,<int>,<dbl>,<dbl>
1,1306,0,0
2,1304,0,0
3,1298,0,0
4,1293,0,0
5,1290,0,0
6,1288,0,0


In [10]:
dat = animalmodel.dat

In [5]:
dat = animalmodel.dat
ped = animalmodel.ped
dat[dat==0] = NA
str(dat)

'data.frame':	1084 obs. of  6 variables:
 $ ANIMAL: Factor w/ 1084 levels "1","2","3","5",..: 864 1076 549 989 1030 751 987 490 906 591 ...
 $ MOTHER: Factor w/ 429 levels "1","2","3","8",..: 362 268 216 375 396 289 328 255 347 240 ...
 $ BYEAR : Factor w/ 34 levels "968","970","971",..: 1 1 2 2 2 2 3 3 3 3 ...
 $ SEX   : Factor w/ 2 levels "1","2": 1 1 2 1 2 1 2 1 1 1 ...
 $ BWT   : num  10.77 9.3 3.98 5.39 12.12 ...
 $ TARSUS: num  24.8 22.5 12.9 20.5 NA ...


In [6]:
# 对BWT和TARSUS两个性状进行多性状模型分析
## 固定因子：SEX
## 随机因子：ANIMAL 

ainv = ainverse(ped)
mod = asreml(cbind(BWT,TARSUS) ~ trait + trait:SEX,
             random = ~ us(trait):vm(ANIMAL,ainv),
             residual = ~ units:us(trait),
             data=dat)

Model fitted using the sigma parameterization.
ASReml 4.1.0 Sat Dec 24 09:02:58 2022
          LogLik        Sigma2     DF     wall    cpu
 1     -2659.585           1.0   1533 09:02:58    0.0
 2     -2624.450           1.0   1533 09:02:58    0.0
 3     -2592.290           1.0   1533 09:02:58    0.0
 4     -2579.349           1.0   1533 09:02:58    0.0
 5     -2577.230           1.0   1533 09:02:58    0.0
 6     -2577.188           1.0   1533 09:02:58    0.0
 7     -2577.188           1.0   1533 09:02:58    0.0


In [7]:
summary(mod)$varcomp

,component,std.error,z.ratio,bound,%ch
,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
"trait:vm(ANIMAL, ainv)!trait_BWT:BWT",2.986952,0.5218132,5.724180,P,0.0
"trait:vm(ANIMAL, ainv)!trait_TARSUS:BWT",2.494674,0.9920503,2.514665,P,0.1
"trait:vm(ANIMAL, ainv)!trait_TARSUS:TARSUS",12.397268,3.0675900,4.041370,P,0.0
units:trait!R,1.000000,NA,NA,F,0.0
units:trait!trait_BWT:BWT,2.995429,0.4183571,7.159981,P,0.0
units:trait!trait_TARSUS:BWT,3.596542,0.8368501,4.297714,P,0.1
units:trait!trait_TARSUS:TARSUS,17.767578,2.6651323,6.666678,P,0.0


In [8]:
mod$converge

[1] TRUE

In [9]:
vpredict(mod,rg ~ V2/sqrt(V1*V3))

,Estimate,SE
,<dbl>,<dbl>
rg,0.4099554,0.1192531


In [10]:
vpredict(mod,rp ~ (V2+V6)/sqrt((V1+V5)*(V3+V7)))

,Estimate,SE
,<dbl>,<dbl>
rp,0.4534364,0.03175156
